In [ ]:
#Cleaning한 COCO format을 UFO로 변환하는 코드

import json

# COCO 형식의 JSON 파일 불러오기
with open("/data/ephemeral/home/MCG/NewDataGenCode/viet_clean.json", "r") as f:
    coco_data = json.load(f)

# 새 JSON 형식으로 변환
new_format = {
    "images": {},
}

# 이미지 정보 채우기
for image in coco_data["images"]:
    image_id = image["id"]
    new_format["images"][image["file_name"]] = {
        "paragraphs": {},
        "words": {},
        "chars": {},
        "img_w": image["width"],
        "img_h": image["height"],
        "num_patches": None,
        "tags": [],
        "relations": {},
        "annotation_log": {
            "worker": "worker",
            "timestamp": "",
            "tool_version": "",
            "source": None
        },
        "license_tag": {
            "usability": True,
            "public": False,
            "commercial": True,
            "type": None,
            "holder": ""
        }
    }

# 어노테이션 정보 채우기
for annotation in coco_data["annotations"]:
    image_id = annotation["image_id"]
    category_id = annotation["category_id"]
    transcription = ""  # 필요시 실제 텍스트 정보로 수정
    points = annotation["segmentation"][0]
    word_id = str(annotation["id"]).zfill(4)  # ID를 4자리로 패딩

    # 해당 이미지 파일명 찾기
    image_file = next((img["file_name"] for img in coco_data["images"] if img["id"] == image_id), None)
    if image_file:
        # 좌표를 점의 쌍으로 변환
        points_pairs = [[points[i], points[i + 1]] for i in range(0, len(points), 2)]

        # 이미지 내 단어 추가
        new_format["images"][image_file]["words"][word_id] = {
            "transcription": transcription,
            "points": points_pairs
        }

# 변환 결과 저장
with open("viet_clean_ufo.json", "w") as f:
    json.dump(new_format, f, ensure_ascii=False, indent=4)

print("변환 완료!")


변환 완료!


In [ ]:
#변환된 json 파일 검토

import json

# JSON 파일 경로
file_path = '/data/ephemeral/home/MCG/code/data/vietnamese_receipt/ufo/train.json'

def check_points_length(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 포인트 개수가 4가 아닌 words 항목 확인
    irregular_words = {}
    for image_name, image_data in data['images'].items():
        for word_id, word_data in image_data['words'].items():
            points = word_data.get('points', [])
            if len(points) != 4:
                if image_name not in irregular_words:
                    irregular_words[image_name] = {}
                irregular_words[image_name][word_id] = points
    
    # 결과 출력
    if irregular_words:
        print("Found words with irregular point counts:")
        for image_name, words in irregular_words.items():
            print(f"Image: {image_name}")
            for word_id, points in words.items():
                print(f"  Word ID: {word_id}, Points Count: {len(points)}, Points: {points}")
    else:
        print("All words have 4 points.")

# 실행
check_points_length(file_path)


All words have 4 points.
